In [58]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [59]:
df = pd.read_csv("creditcard.csv", delimiter = ",")

In [60]:
df.drop(["Time"], axis=1, inplace = True)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [61]:
scaler = StandardScaler()
df["standardisedTime"] = scaler.fit_transform(df["Amount"].values.reshape(-1, 1))
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,standardisedTime
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,-0.073403


In [62]:
df.drop(["Amount"], axis=1, inplace = True)

In [63]:
y = df["Class"]
x = df.drop(["Class"], axis = 1)

In [64]:
x

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,standardisedTime
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.350151
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,-0.254117
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,-0.081839
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.313249


In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [66]:
model = Sequential([
Dense(input_dim = 29, units = 16, activation = "relu"),
Dense(units = 24, activation = "relu"),
Dropout(0.5),
Dense(units = 20, activation = "relu"),
Dense(units = 24, activation = "relu"),
Dense(units =1, activation = "sigmoid")])

In [67]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(x_train, y_train, batch_size = 15, epochs = 5)

Train on 199364 samples
Epoch 1/5
199364/199364 [==============================] - 20s 99us/sample - loss: 0.0089 - accuracy: 0.9983
Epoch 2/5
199364/199364 [==============================] - 21s 105us/sample - loss: 0.0046 - accuracy: 0.9993
Epoch 3/5
199364/199364 [==============================] - 19s 93us/sample - loss: 0.0040 - accuracy: 0.9993
Epoch 4/5
199364/199364 [==============================] - 20s 102us/sample - loss: 0.0039 - accuracy: 0.9994
Epoch 5/5
199364/199364 [==============================] - 20s 99us/sample - loss: 0.0034 - accuracy: 0.9994


In [68]:
y_pred = model.predict(x_test)


In [69]:
y_pred = np.ravel(y_pred)
y_pred.shape

(85443,)

In [70]:
thresh = 0.5
for i in range(len(y_pred)):
    if(y_pred[i] >= thresh):
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [71]:
confusion_matrix(np.array(y_test).reshape(-1, 1), y_pred)

array([[85251,    19],
       [   30,   143]], dtype=int64)

In [72]:
precision_recall_fscore_support(np.array(y_test).reshape(-1, 1), y_pred, average = "binary")

(0.8827160493827161, 0.8265895953757225, 0.853731343283582, None)

## classification using SMOTE

In [73]:
from imblearn.over_sampling import SMOTE
from collections import Counter
smote = SMOTE()


In [81]:
x_smote, y_smote = smote.fit_resample(x, y)

print('Original dataset shape', Counter(y_train))
print('Resample dataset shape', Counter(y_smote))

Original dataset shape Counter({0: 199045, 1: 319})
Resample dataset shape Counter({0: 284315, 1: 284315})


In [82]:
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size = 0.3)

In [83]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(x_train, y_train, batch_size = 15, epochs = 5)

Train on 398041 samples
Epoch 1/5
398041/398041 [==============================] - 46s 116us/sample - loss: 0.0191 - accuracy: 0.9950
Epoch 2/5
398041/398041 [==============================] - 41s 104us/sample - loss: 0.0096 - accuracy: 0.9977
Epoch 3/5
398041/398041 [==============================] - 43s 108us/sample - loss: 0.0085 - accuracy: 0.9980
Epoch 4/5
398041/398041 [==============================] - 40s 100us/sample - loss: 0.0080 - accuracy: 0.9982
Epoch 5/5
398041/398041 [==============================] - 39s 99us/sample - loss: 0.0073 - accuracy: 0.9984


In [84]:
y_pred = model.predict(x_test)

In [85]:
y_pred = np.ravel(y_pred)
y_pred.shape

(170589,)

In [86]:
thresh = 0.5
for i in range(len(y_pred)):
    if(y_pred[i] >= thresh):
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [87]:
confusion_matrix(np.array(y_test).reshape(-1, 1), y_pred)

array([[84832,   184],
       [   75, 85498]], dtype=int64)

In [90]:
precision_recall_fscore_support(np.array(y_test).reshape(-1, 1), y_pred, average = "binary")

(0.9978525244508765, 0.9991235553270307, 0.9984876353975067, None)